In [1]:
import seaborn as sns
import pandas as pd
import numpy as np 
import re
pd.set_option('display.max_columns', 500)

In [2]:
#open csv, drop unnecessary columns
nba = pd.read_csv("../nba.csv")
nba = nba.drop(columns = ["Unnamed: 6", "Start (ET)","Notes","Unnamed: 7","Attend."])


In [3]:
#clean date and turn to date_time type
nba.Date = nba.Date.str.replace("^[A-z]{3}","-")
nba.Date.str.lstrip("- ")
nba.Date = nba.Date.str.replace(" ","-").str.lstrip("-")
nba.Date = pd.to_datetime(nba.Date)

In [4]:
#renaming columns
nba = nba.rename(columns = {"PTS":"AwayPTS", "PTS.1":"HomePTS", "Visitor/Neutral":"Away","Home/Neutral":"Home", "Attend.":"Attend"})

In [5]:
nba.head()

,Date,Away,AwayPTS,Home,HomePTS
0,2018-10-16,Philadelphia 76ers,87,Boston Celtics,105
1,2018-10-16,Oklahoma City Thunder,100,Golden State Warriors,108
2,2018-10-17,Milwaukee Bucks,113,Charlotte Hornets,112
3,2018-10-17,Brooklyn Nets,100,Detroit Pistons,103
4,2018-10-17,Memphis Grizzlies,83,Indiana Pacers,111


In [6]:
#creating homewins columns
nba["HomeWin"] = np.where(nba["HomePTS"] > nba["AwayPTS"], 1,0)


In [7]:
# timeawaytest = nba.drop(columns = ["Date","AwayPTS","HomePTS"])
# timeawaytest["TimeAway"] = nbatest["Timeaway"]
# timeawaytest["HomeWinStreak"] = nbatest["HomeWinStreak"]
# timeawaytest["AwayWinStreak"] = nbatest["AwayWinStreak"]
# # timeawaytest[["AwayWinStreak","HomeLoseStreak","AwayLoseStreak","HomeCoachSavage","AwayCoachSavage"]]=nbatest[["AwayWinStreak","HomeLoseStreak","AwayLoseStreak","HomeCoachSavage","AwayCoachSavage"]]
# # timeawaytest[["HomeAllStars","AwayAllStars"]] = nbatest[["HomeAllstars","AwayAllstars"]]


# timeawaytest

In [8]:
nbatest = nba[["Away","Home","HomeWin"]]


In [9]:
from sklearn import preprocessing
dummies = pd.get_dummies(nbatest[["Away","Home"]])
nbatest[dummies.columns] = dummies

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:3489: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [10]:
nbatest = nbatest.drop(columns = ["Home","Away"])
nbatest

,HomeWin,Away_Atlanta Hawks,Away_Boston Celtics,Away_Brooklyn Nets,Away_Charlotte Hornets,Away_Chicago Bulls,Away_Cleveland Cavaliers,Away_Dallas Mavericks,Away_Denver Nuggets,Away_Detroit Pistons,Away_Golden State Warriors,Away_Houston Rockets,Away_Indiana Pacers,Away_Los Angeles Clippers,Away_Los Angeles Lakers,Away_Memphis Grizzlies,Away_Miami Heat,Away_Milwaukee Bucks,Away_Minnesota Timberwolves,Away_New Orleans Pelicans,Away_New York Knicks,Away_Oklahoma City Thunder,Away_Orlando Magic,Away_Philadelphia 76ers,Away_Phoenix Suns,Away_Portland Trail Blazers,Away_Sacramento Kings,Away_San Antonio Spurs,Away_Toronto Raptors,Away_Utah Jazz,Away_Washington Wizards,Home_Atlanta Hawks,Home_Boston Celtics,Home_Brooklyn Nets,Home_Charlotte Hornets,Home_Chicago Bulls,Home_Cleveland Cavaliers,Home_Dallas Mavericks,Home_Denver Nuggets,Home_Detroit Pistons,Home_Golden State Warriors,Home_Houston Rockets,Home_Indiana Pacers,Home_Los Angeles Clippers,Home_Los Angeles Lakers,Home_Memphis Grizzlies,Home_Miami Heat,Home_Milwaukee Bucks,Home_Minnesota Timberwolves,Home_New Orleans Pelicans,Home_New York Knicks,Home_Oklahoma City Thunder,Home_Orlando Magic,Home_Philadelphia 76ers,Home_Phoenix Suns,Home_Portland Trail Blazers,Home_Sacramento Kings,Home_San Antonio Spurs,Home_Toronto Raptors,Home_Utah Jazz,Home_Washington Wizards
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [11]:
#defining functions that check the accuracy score of our predictions using different models
#Random Forests
def RFscore(nbatest):
    X_train = nbatest[:984].drop(columns = "HomeWin")    
    y_train = nbatest["HomeWin"][:984]
    X_test = nbatest[984:].drop(columns = "HomeWin")
    y_test = nbatest["HomeWin"][984:]
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.datasets import make_classification
    mod = RandomForestClassifier(n_estimators = 100)
    mod.fit(X_train, y_train)
    y_pred = mod.predict(X_test)
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_test,y_pred)


#SVC
def SVCscore(nbatest):
    #train data/ test data
    X_train = nbatest[:984].drop(columns = "HomeWin")    
    y_train = nbatest["HomeWin"][:984]
    X_test = nbatest[984:].drop(columns = "HomeWin")
    y_test = nbatest["HomeWin"][984:]
    
    #fit model
    from sklearn.svm import SVC
    svc = SVC(gamma='auto')
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_test,y_pred)


#LogisticRegression
def LRscore(nbatest):
    X_train = nbatest[:984].drop(columns = "HomeWin")    
    y_train = nbatest["HomeWin"][:984]
    X_test = nbatest[984:].drop(columns = "HomeWin")
    y_test = nbatest["HomeWin"][984:]
    
    from sklearn.linear_model import LogisticRegression
    mod = LogisticRegression()
    mod.fit(X_train,y_train)
    y_pred = mod.predict(X_test)
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_test,y_pred)

In [12]:
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.6747967479674797 
SVC Accuracy score: 0.5934959349593496 
Random Forests Accuracy score: 0.6504065040650406


In [13]:
#Adding "time on the road" feature
nbatest["Timeaway"] = 0
away_counts = {}
for i in nba["Away"].unique():
    away_counts[i] = 0
for i in range(len(nba.Away)):
    away_counts[nba.Away[i]] +=1
    away_counts[nba.Home[i]] = 0
    nbatest.iloc[i,-1] = away_counts[nba.Away[i]]

print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.6788617886178862 
SVC Accuracy score: 0.5934959349593496 
Random Forests Accuracy score: 0.6382113821138211


In [14]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["Timeaway"]])
scaled = scaler.transform(nbatest[["Timeaway"]])
nbatest[["Timeaway"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.6788617886178862 
SVC Accuracy score: 0.5934959349593496 
Random Forests Accuracy score: 0.6544715447154471


In [15]:
#adding Home win streak and Away win streak features
nbatest["HomeWinStreak"] = 0
nbatest["AwayWinStreak"] = 0
win_counts = {}
for i in nba["Away"].unique():
    win_counts[i] = 0
    
for row in range(len(nba)):
    nbatest.iloc[row,-2] = win_counts[nba["Home"][row]]
    nbatest.iloc[row,-1] = win_counts[nba["Away"][row]]
    if nbatest["HomeWin"][row] == 1:
        win_counts[nba["Home"][row]] +=1
        win_counts[nba["Away"][row]] == 0
    else:
        win_counts[nba["Away"][row]] +=1
        win_counts[nba["Home"][row]] == 0

    
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.6788617886178862 
SVC Accuracy score: 0.6747967479674797 
Random Forests Accuracy score: 0.6585365853658537


In [16]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["HomeWinStreak","AwayWinStreak"]])
scaled = scaler.transform(nbatest[["HomeWinStreak","AwayWinStreak"]])
nbatest[["HomeWinStreak","AwayWinStreak"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.6788617886178862 
SVC Accuracy score: 0.6585365853658537 
Random Forests Accuracy score: 0.6504065040650406


In [17]:
def SVCrandscore(nbatest):
    #train data/ test data
    from sklearn.model_selection import train_test_split
    X = nbatest.drop(columns = "HomeWin")
    y = nbatest["HomeWin"]
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.8)
    
    #fit model
    from sklearn.svm import SVC
    svc = SVC(gamma='auto')
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    from sklearn.metrics import accuracy_score
    return accuracy_score(y_test,y_pred)


In [18]:
#adding Home lose streak and Away lose streak features
nbatest["HomeLoseStreak"] = 0
nbatest["AwayLoseStreak"] = 0
loss_counts = {}
for i in nba["Away"].unique():
    loss_counts[i] = 0
    

for row in range(len(nba)):
    if nbatest["HomeWin"][row] == 1:
        loss_counts[nba["Away"][row]] +=1
        loss_counts[nba["Home"][row]] == 0
    else:
        loss_counts[nba["Home"][row]] +=1
        loss_counts[nba["Away"][row]] == 0
    nbatest.iloc[row,-2] = loss_counts[nba["Home"][row]]
    nbatest.iloc[row,-1] = loss_counts[nba["Away"][row]]
    
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.8130081300813008 
SVC Accuracy score: 0.6463414634146342 
Random Forests Accuracy score: 0.6422764227642277


In [19]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["HomeLoseStreak","AwayLoseStreak"]])
scaled = scaler.transform(nbatest[["HomeLoseStreak","AwayLoseStreak"]])
nbatest[["HomeLoseStreak","AwayLoseStreak"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.7642276422764228 
SVC Accuracy score: 0.7113821138211383 
Random Forests Accuracy score: 0.5609756097560976


In [20]:
nbatest["HomeCoachSavage"] = 0
nbatest["AwayCoachSavage"] = 0

nbatest.loc[nba["Home"]=="San Antonio Spurs","HomeCoachSavage"] = 6
nbatest.loc[nba["Home"]=="Boston Celtics","HomeCoachSavage"] = 5
nbatest.loc[nba["Home"]=="Golden State Warriors","HomeCoachSavage"] = 4
nbatest.loc[nba["Home"]=="Utah Jazz","HomeCoachSavage"] = 3
nbatest.loc[nba["Home"]=="Houston Rockets","HomeCoachSavage"] = 2
nbatest.loc[nba["Home"]=="Toronto Raptors","HomeCoachSavage"] = 1



nbatest.loc[nba["Away"]=="San Antonio Spurs","AwayCoachSavage"] = 6
nbatest.loc[nba["Away"]=="Boston Celtics","AwayCoachSavage"] = 5
nbatest.loc[nba["Away"]=="Golden State Warriors","AwayCoachSavage"] = 4
nbatest.loc[nba["Away"]=="Utah Jazz","AwayCoachSavage"] = 3
nbatest.loc[nba["Away"]=="Houston Rockets","AwayCoachSavage"] = 2
nbatest.loc[nba["Away"]=="Toronto Raptors","AwayCoachSavage"] = 1


print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.7764227642276422 
SVC Accuracy score: 0.7113821138211383 
Random Forests Accuracy score: 0.6463414634146342


In [21]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["HomeCoachSavage","AwayCoachSavage"]])
scaled = scaler.transform(nbatest[["HomeCoachSavage","AwayCoachSavage"]])
nbatest[["HomeCoachSavage","AwayCoachSavage"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.7764227642276422 
SVC Accuracy score: 0.7032520325203252 
Random Forests Accuracy score: 0.6341463414634146


In [22]:
allstar_count = {}
for i in nba.Home.unique():
    allstar_count[i] = 0
allstar_count['Philadelphia 76ers'] = 2
allstar_count['Milwaukee Bucks'] = 2
allstar_count['Oklahoma City Thunder']=2
allstar_count['Golden State Warriors'] = 3
allstar_count['Denver Nuggets'] = 1
allstar_count['Detroit Pistons'] = 1
allstar_count['Brooklyn Nets'] = 1
allstar_count['Orlando Magic'] = 1
allstar_count['Toronto Raptors'] = 2
allstar_count['Dallas Mavericks'] = 1
allstar_count['Los Angeles Lakers'] = 1
allstar_count['Houston Rockets'] = 1
allstar_count['Orlando Magic'] = 1
allstar_count['Boston Celtics'] = 1
allstar_count['New Orleans Pelicans'] = 1
allstar_count[ 'Portland Trail Blazers'] = 1
allstar_count['San Antonio Spurs'] = 1
allstar_count['Minnesota Timberwolves'] = 1
allstar_count['Washington Wizards'] = 1
allstar_count['Miami Heat'] = 1


In [23]:
nbatest["HomeAllstars"] = 0
nbatest["AwayAllstars"] = 0
for i in range(len(nba)):
    nbatest.iloc[i,-1] = allstar_count[nba["Away"][i]]
    nbatest.iloc[i,-2] = allstar_count[nba["Home"][i]]


In [24]:
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.7764227642276422 
SVC Accuracy score: 0.7032520325203252 
Random Forests Accuracy score: 0.6788617886178862


In [25]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["HomeAllstars","AwayAllstars"]])
scaled = scaler.transform(nbatest[["HomeAllstars","AwayAllstars"]])
nbatest[["HomeAllstars","AwayAllstars"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.7764227642276422 
SVC Accuracy score: 0.7073170731707317 
Random Forests Accuracy score: 0.6869918699186992


In [26]:
depth = pd.read_csv("depth.csv")
depth["avg"] = (depth["bench1"]+depth["bench2"]+depth["bench3"]+depth["bench4"]+depth["bench5"])/5

# depth

In [27]:
depth.sort_values("avg")

,team,bench1,bench2,bench3,bench4,bench5,avg
6,Portland Trail Blazers,73,75,70,70,70,71.6
13,Chicago Bulls,69,74,70,77,71,72.2
28,New Orleans Pelicans,70,75,73,72,73,72.6
14,Cleveland Cavaliers,73,75,68,74,76,73.2
5,Denver Nuggets,70,72,76,73,76,73.4
9,Minnesota Timberwolves,75,76,75,71,70,73.4
24,Atlanta Hawks,75,70,72,77,74,73.6
12,Detroit Pistons,76,69,75,72,76,73.6
29,Memphis Grizzlies,77,74,66,73,78,73.6
19,Phoenix Suns,70,77,74,75,73,73.8


In [28]:
depth_dict ={}
for i in depth.team.unique():
    depth_dict[i] = float(depth[depth["team"] == i]["avg"])
nba.head()
print(depth_dict, nba.head())

{'Toronto Raptors': 75.6, 'Philadelphia 76ers': 75.4, 'Boston Celtics': 79.6, 'Brooklyn Nets': 76.2, 'New York Knicks': 75.6, 'Denver Nuggets': 73.4, 'Portland Trail Blazers': 71.6, 'Utah Jazz': 74.2, 'Oklahoma City Thunder': 74.4, 'Minnesota Timberwolves': 73.4, 'Milwaukee Bucks': 75.4, 'Indiana Pacers': 75.6, 'Detroit Pistons': 73.6, 'Chicago Bulls': 72.2, 'Cleveland Cavaliers': 73.2, 'Golden State Warriors': 74.0, 'Los Angeles Clippers': 77.6, 'Sacramento Kings': 74.8, 'Los Angeles Lakers': 77.6, 'Phoenix Suns': 73.8, 'Orlando Magic': 74.2, 'Miami Heat': 77.2, 'Charlotte Hornets': 75.0, 'Washington Wizards': 74.8, 'Atlanta Hawks': 73.6, 'Houston Rockets': 78.0, 'San Antonio Spurs': 74.0, 'Dallas Mavericks': 75.0, 'New Orleans Pelicans': 72.6, 'Memphis Grizzlies': 73.6}         Date                   Away  AwayPTS                   Home  HomePTS  \
0 2018-10-16     Philadelphia 76ers       87         Boston Celtics      105   
1 2018-10-16  Oklahoma City Thunder      100  Golden Stat

In [29]:
nbatest["HomeBenchRating"] = 0
nbatest["AwayBenchRating"] = 0
for i in range(len(nba)):
    nbatest.iloc[i,-1] = depth_dict[nba.Away[i]]
    nbatest.iloc[i,-2] = depth_dict[nba.Home[i]]


In [30]:
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.7804878048780488 
SVC Accuracy score: 0.7073170731707317 
Random Forests Accuracy score: 0.6544715447154471


In [31]:
nbatest.head(10)

,HomeWin,Away_Atlanta Hawks,Away_Boston Celtics,Away_Brooklyn Nets,Away_Charlotte Hornets,Away_Chicago Bulls,Away_Cleveland Cavaliers,Away_Dallas Mavericks,Away_Denver Nuggets,Away_Detroit Pistons,Away_Golden State Warriors,Away_Houston Rockets,Away_Indiana Pacers,Away_Los Angeles Clippers,Away_Los Angeles Lakers,Away_Memphis Grizzlies,Away_Miami Heat,Away_Milwaukee Bucks,Away_Minnesota Timberwolves,Away_New Orleans Pelicans,Away_New York Knicks,Away_Oklahoma City Thunder,Away_Orlando Magic,Away_Philadelphia 76ers,Away_Phoenix Suns,Away_Portland Trail Blazers,Away_Sacramento Kings,Away_San Antonio Spurs,Away_Toronto Raptors,Away_Utah Jazz,Away_Washington Wizards,Home_Atlanta Hawks,Home_Boston Celtics,Home_Brooklyn Nets,Home_Charlotte Hornets,Home_Chicago Bulls,Home_Cleveland Cavaliers,Home_Dallas Mavericks,Home_Denver Nuggets,Home_Detroit Pistons,Home_Golden State Warriors,Home_Houston Rockets,Home_Indiana Pacers,Home_Los Angeles Clippers,Home_Los Angeles Lakers,Home_Memphis Grizzlies,Home_Miami Heat,Home_Milwaukee Bucks,Home_Minnesota Timberwolves,Home_New Orleans Pelicans,Home_New York Knicks,Home_Oklahoma City Thunder,Home_Orlando Magic,Home_Philadelphia 76ers,Home_Phoenix Suns,Home_Portland Trail Blazers,Home_Sacramento Kings,Home_San Antonio Spurs,Home_Toronto Raptors,Home_Utah Jazz,Home_Washington Wizards,Timeaway,HomeWinStreak,AwayWinStreak,HomeLoseStreak,AwayLoseStreak,HomeCoachSavage,AwayCoachSavage,HomeAllstars,AwayAllstars,HomeBenchRating,AwayBenchRating
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.79521,-1.480008,-1.476939,-1.490353,-1.451275,2.696291,-0.438931,0.214176,1.499235,79.6,75.4
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.79521,-1.480008,-1.476939,-1.490353,-1.451275,2.069247,-0.438931,2.784294,1.499235,74.0,74.4
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.79521,-1.480008,-1.476939,-1.418617,-1.524962,-0.438931,-0.438931,-1.070882,1.499235,75.0,75.4
3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.79521,-1.480008,-1.476939,-1.490353,-1.451275,-0.438931,-0.438931,0.214176,0.214176,73.6,76.2
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.79521,-1.480008,-1.476939,-1.490353,-1.451275,-0.438931,-0.438931,-1.070882,-1.070882,75.6,73.6
5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-0.79521,-1.480008,-1.476939,-1.490353,-1.451275,-0.438931,-0.438931,0.214176,0.214176,74.2,77.2
6,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-0.79521,-1.480008,-1.476939,-1.490353,-1.451275,-0.438931,-0.438931,-1.070882,-1.070882,75.6,73.6
7,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-0.79521,-1.480008,-1.476939,-1.490353,-1.451275,0.188113,-0.438931,1.499235,-1.070882,75.6,73.2
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.79521,-1.480008,-1.476939,-1.418617,-1.524962,0.815158,-0.438931,0.214176,0.214176,78.0,72.6
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-0.79521,-1.480008,-1.476939,-1.490353,-1.451275,3.323336,-0.438931,0.214176,0.214176,74.0,73.4


In [32]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["AwayBenchRating","HomeBenchRating"]])
scaled = scaler.transform(nbatest[["AwayBenchRating","HomeBenchRating"]])
nbatest[["AwayBenchRating","HomeBenchRating"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Accuracy Score:  0.7764227642276422 
SVC Accuracy score: 0.7073170731707317 
Random Forests Accuracy score: 0.6504065040650406


In [ ]:
score_dict = {}
for i in nba.Home.unique():
    score_dict[i] = {"counter" : 0, "totalscore" : 0}
score_dict["Miami Heat"]

In [ ]:
nbatest["AwayAvgScore"] = 0
nbatest["HomeAvgScore"] = 0
for i in range(len(nba)):
    #Home team counter and total score
    score_dict[nba.Home[i]]["counter"] +=1 
    score_dict[nba.Home[i]]["totalscore"] += nba.HomePTS[i]
    #Away team counter and score
    score_dict[nba.Away[i]]["counter"]+= 1 
    score_dict[nba.Away[i]]["totalscore"] += nba.AwayPTS[i]
    
    nbatest.iloc[i,-1] = score_dict[nba.Home[i]]["totalscore"] / score_dict[nba.Home[i]]["counter"]
    nbatest.iloc[i,-2] = score_dict[nba.Away[i]]["totalscore"] / score_dict[nba.Away[i]]["counter"]
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["AwayAvgScore","HomeAvgScore"]])
scaled = scaler.transform(nbatest[["AwayAvgScore","HomeAvgScore"]])
nbatest[["AwayAvgScore","HomeAvgScore"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


In [ ]:
sns.heatmap(nbatest[['Timeaway', 'HomeWinStreak', 'AwayWinStreak',
       'HomeLoseStreak', 'AwayLoseStreak', 'HomeCoachSavage',
       'AwayCoachSavage', 'HomeAllstars', 'AwayAllstars',"HomeBenchRating","AwayBenchRating"]].corr())

In [ ]:
#RANKINGS
#Creates dictionary with team and their rankings
feb_ranks = pd.read_csv("feb_ranks.csv")
feb_ranks["Team"] = feb_ranks["Western Conference"]
rankdict = {}
for i in range(len(feb_ranks.Team)):
    rankdict[feb_ranks.Team[i]] = feb_ranks.Rk[i]


In [ ]:
#Creates HomeRank and AwayRank for each matchup in nba dataset
nbatest["HomeRank"] = 0
nbatest["AwayRank"] = 0
for i in range(len(nbatest.HomeRank)):
    #Setting AwayRank for row i
    nbatest.iloc[i,-1] = rankdict[nba.Away[i]]
    #Setting HomeRank for row i
    nbatest.iloc[i,-2] = rankdict[nba.Home[i]]

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(nbatest[["HomeRank","AwayRank"]])
scaled = scaler.transform(nbatest[["HomeRank","AwayRank"]])
nbatest[["HomeRank","AwayRank"]]= scaled
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))


In [ ]:
#creates a single column that checks if the home team is ranked higher than away team 
rankspread = nbatest["HomeRank"]-nbatest["AwayRank"]
nbatest["HomeRanksHigher"]= 0
#if rankspread is positive, home is better
#if rankspread is negative, away is better
for i in range(len(nbatest)):
    if rankspread[i] > 0:
        nbatest.iloc[i,-1] = 1
    else:
        nbatest.iloc[i,-1] = 0
        

print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))

In [ ]:
#creates  a single column that checks if the home team is significantly better than the away team
rankspread = nbatest["HomeRank"]-nbatest["AwayRank"]

nbatest["HomeisFav"]= 0
for i in range(len(nbatest)):
    if rankspread[i] > 0 and rankspread[i] >5:
        nbatest.iloc[i,-1] = 1
    elif rankspread[i]<0 and abs(rankspread[i])>5:
        nbatest.iloc[i,-1] = 0
    else:
        nbatest.iloc[i,-1] = 0
nbatest.drop(columns = ["HomeRank","AwayRank"], inplace = True) 
print("Logistic Regression Accuracy Score: ",LRscore(nbatest),"\nSVC Accuracy score:",SVCscore(nbatest),"\nRandom Forests Accuracy score:",RFscore(nbatest))

In [ ]:

nbatest[["HomeWinStreak","AwayWinStreak"]]